<img src="Images/SunSolveLogo.svg">

# SunSolve P90 Analysis Notebook

[SunSolve P90 documentation](https://docs.sunsolve.com/en/p90/getting-started/jupyter-notebook-guide/)

[SunSolve P90 signup form](https://www.sunsolve.com/p90/signup/) (mandatory to gain access)

[Jupyter notebook tutorial](https://colab.research.google.com/github/pvlighthouse/SunSolveP90/blob/main/Jupyter%20Notebook%20Intro.ipynb)

## 1. Install libraries and login to SunSolve P90

In [ ]:
print("Installing SunSolve P90 client. The pip dependency error can be safely ignored (relates to Colab's preinstalled packages).")
%pip install https://github.com/pvlighthouse/SunSolveP90/raw/refs/heads/main/sunsolve_p90_client-0.1.0.170+dev-py3-none-any.whl --quiet
import os, sys
from pvl_p90_client.client.p90_client import P90Client
from pvl_p90_client.helpers import pvl_login

with P90Client(version="latest-dev") as client:
    # User will be prompted to enter credentials
    credentials = pvl_login.login()
print("Logged in to SunSolve P90.")

## 2. Load helper functions (and optionally connect to Google Drive)

In [ ]:
USING_DRIVE = False  # Set to True if using Google Colab and Drive. Set DRIVE_FOLDER below to the target sub-folder of your drive.
DRIVE_FOLDER = r"/content/G/MyDrive/"    # Windows paths (G:\\ like) will be handled below, Mac paths (~/Google Drive/) also. Alternatively, write as "/content/G/MyDrive/..." or "/content/G/SharedDrives/...".

# Set up Drive folder if using Colab + Drive (must enter DRIVE_FOLDER path above)
if USING_DRIVE and "google.colab" in sys.modules:
    from google.colab import drive
    folder = DRIVE_FOLDER.replace("\\", "/").replace("G://", "/content/G/").replace("~/Google Drive/", "/content/G/").replace("/My Drive", "/MyDrive").replace("/Shared Drives", "/SharedDrives")    # Colab doesn't use spaces for these base folder names
    drive.mount("/content/G")   # Mount Google Drive to "/content/G" (A pop-up will ask for authorization, you must press continue twice but don't need to allow additional permissions)
    os.chdir(folder)            # Change the working directory to your project folder

# Check if the helper file is available, or else clone the repo into a "SunSolve P90" sub-folder (new project folder)
if not "UncertaintyFunctions.ipynb" in os.listdir():
    if not "SunSolve P90" in os.listdir() or not "UncertaintyFunctions.ipynb" in os.listdir("SunSolve P90"):
        os.makedirs("SunSolve P90", exist_ok=True)
        os.system("git clone --depth 1 https://github.com/SF-PVL/P90-Notebook.git 'SunSolve P90'")
    os.rename(".pvlToken", "SunSolve P90/.pvlToken")    # Move .pvlToken to SunSolve P90 directory.
    os.chdir("SunSolve P90")    # Make the new "SunSolve P90" folder the project folder

# Load helper functions
%run "UncertaintyFunctions.ipynb"   # If file cannot be found, try restarting the Python kernel and re-running cells 1 and 2. Top menu `Restart` or `Runtime → Restart → Restart runtime`
print("Helper functions loaded.")

## 3. Load inputs

[Available Inputs](https://docs.sunsolve.com/en/p90/simulation-inputs)

In [ ]:
# Set uncertainty simulation constants (time scales with N_SIMS*N_YEARS, 5000*10 ≈ 120 seconds)
simulation_options = build_simulation_options(
    number_of_years=5,
    number_of_simulations=5000
)

# Load weather: sydney.pvw file is an example weather file
weather_file_path = "Data/sydney.pvw"
weather_data = load_weather_data_from_pvw_file(weather_file_path)

# Assign specific system inputs
optical_settings = build_optical_settings(
    fallback_soiling_front=1
)

operational_settings = build_operational_settings(
    availability=1
)

# Use defaults for other inputs
module_info = build_module_info()
system_info = build_system_info()
electrical_settings = build_electrical_settings()
thermal_settings = build_thermal_settings()

# Set output options
result_options = build_result_options(
    p_min=0.9,          # Minimum relative yield for histogram bins (90% of P50)
    p_delta=0.01,       # 1% bin widths (relative to P50)
    p_values=[5, 10, 90, 95]   # Request P90 and P95 results
)

# Print completion message
print("Setup complete.")

## 4. Plot weather (optional)

In [ ]:
interactive_weather_plot(weather_file_path, weather_data)

## 5. Visualise uncertainty distributions (optional)

[Options for distribution types](https://docs.sunsolve.com/en/p90/uncertainty-distributions/pdf-types/)

In [ ]:
interactive_distribution_plot()

## 6. Load uncertainty distributions

[Distribution syntax](https://docs.sunsolve.com/en/p90/uncertainty-distributions/syntax/)

<details>
<summary><strong>Inputs that can have distributions</strong></summary>
GHI, DiffuseFraction, WindSpeed, Temperature, ModulePower, SpectralCorrection, SoilingFront, SoilingRear, Uc, Uv, Alpha, AnnualDegradationRate, Availability, YieldModifier, CircumsolarFraction, UndulatingGround, ExtraIrradiance, Curtailment, DCHealth, InverterToInverterMismatch, StringToStringMismatch, ModuleToModuleMismatch, CellToCellMismatch, InverterEfficiency, ModuleEfficiencyTemperatureCoefficient, Albedo, RearStructuralShadingFactor, RearTransmissionFactor
</details>

In [ ]:
distribution_list = [
    create_distribution(DistributionInput.AnnualDegradationRate, simToSim=["SkewedGaussian", 1, 0.002, 6], yearToYear=["Gaussian", 1, 0.02]),
    create_distribution(DistributionInput.GHI, simToSim=["Gaussian", 1, 0.06], yearToYear=["Gaussian", 1, 0.04], stepToStep=["SkewedGaussian", 0.95, 0.05, -2]),
    create_distribution(DistributionInput.Temperature, simToSim=["Gaussian", 1, 0.06], yearToYear=["Gaussian", 1, 0.04], stepToStep=["SkewedGaussian", 0.95, 0.05, -2]),
    create_distribution(DistributionInput.SoilingFront, simToSim=["Weibull", 0, 0.04, 2], yearToYear=["Gaussian", 0.08, 0.04]),
]
print("Distribution list created.")

## 7. Run analysis

In [ ]:
# Build and send request
p90_request = build_request(
    time_step_data=weather_data,
    module=module_info,
    system=system_info,
    electrical=electrical_settings,
    optical=optical_settings,
    thermal=thermal_settings,
    operational=operational_settings,
    distributions=distribution_list,
    simulation_options=simulation_options,
    result_options=result_options
)

summary, used_inputs = request_analysis(p90_request)
# Convert summary to dataframes and show yearly P-values
yearly_hist_df, yearly_p_df, yearly_p50_df = summary_to_dataframes(summary, used_inputs)

## 8. Results — table of P-values

In [ ]:
yearly_p_df

## 9. Results — histogram

In [ ]:
year_one_yield = 1              # Change this to plot absolute yields, if you know what the first year yield should be in MWh
if used_inputs.SimulationOptions.NumberOfYears > 1:
    plot_yearly_P50_Values(summary, year_one_yield=year_one_yield)  # Plots degradation over N_YEARS, don't plot for single year
plot_interactive_histogram(summary, used_inputs, year_one_yield=year_one_yield)

## 10. Results — P-values vs year

In [ ]:
plot_pvalues(summary, year_one_yield=year_one_yield)

## 11. Export

In [ ]:
CREATE_NOTEBOOK_COPY = False  # Set to True to create a copy of the current notebook with results saved (interactive plots will be lost in the saved copy)
OUTPUT_FOLDER = "P90 Results"   # Folder to save results into, set to "" to save to base folder
if not OUTPUT_FOLDER in os.listdir():
    os.makedirs(OUTPUT_FOLDER)
if not OUTPUT_FOLDER.endswith("/"):
    OUTPUT_FOLDER += "/"

date_prefix = datetime.date.today().strftime("%y%m%d_")
n_sims = used_inputs.SimulationOptions.NumberOfSimulations
n_years = used_inputs.SimulationOptions.NumberOfYears
filename = date_prefix + f"P90 summary_{n_years}x{n_sims}"
# Export to Excel
export_to_excel(summary, OUTPUT_FOLDER+filename, used_inputs=used_inputs, dist_list=distribution_list)

if CREATE_NOTEBOOK_COPY:
    # Create a copy of the current notebook
    result_notebook = OUTPUT_FOLDER+filename.replace("summary", "result") + ".ipynb"
    export_notebook("P90 Analysis.ipynb", result_notebook)

## 12. Further analysis

In [ ]:
# Example: Plot a box plot of all years' relative yield distributions
fig = px.box(yearly_hist_df)
fig.update_layout(
    font=dict(size=16),
    title_font=dict(size=18),
    title=f"Relative yield distribution box plot{' (all-years)' if n_years > 1 else '(single year)'}",
    xaxis_title="Yield (relative to yearly P50)",
    yaxis_title="Frequency",
    width=900,
    paper_bgcolor = "rgba(0,0,0,0)",  # Fully transparent background
    plot_bgcolor = "rgba(0,0,0,0)"
)
fig.update_yaxes(linewidth=1, linecolor="black", mirror=True, ticks="outside", showgrid=True, gridwidth=0.1, gridcolor="#c0c0c0")
fig.update_xaxes(dtick=5, linewidth=1, linecolor="black", mirror=True, ticks="outside")
fig.show()